# 🧬 Articuno Example: Pandas with PyArrow columns

This example demonstrates using Articuno to infer a Pydantic model from a Pandas DataFrame using PyArrow extension dtypes.

In [ ]:
import pandas as pd
from articuno import generate_class_code, infer_pydantic_model


## ✅ Create a Pandas DataFrame with PyArrow-backed columns

In [2]:
df = pd.DataFrame({
    "id": pd.Series([1, 2, 3], dtype="int64[pyarrow]"),
    "name": pd.Series(["Alice", "Bob", "Charlie"], dtype="string[pyarrow]"),
    "score": pd.Series([95.5, 88.0, 92.3], dtype="float64[pyarrow]"),
    "metadata": [
        {"level": 5, "active": True},
        {"level": 3, "active": False},
        {"level": None, "active": True}
    ]
})

df

,id,name,score,metadata
0,1,Alice,95.5,"{'level': 5, 'active': True}"
1,2,Bob,88.0,"{'level': 3, 'active': False}"
2,3,Charlie,92.3,"{'level': None, 'active': True}"


## 🔍 Infer a Pydantic model from this DataFrame

In [6]:
Model = infer_pydantic_model(df, model_name="UserWithArrow", force_optional=True)
print(Model.schema_json(indent=2))

{
  "$defs": {
    "metadata_NestedModel": {
      "properties": {
        "active": {
          "anyOf": [
            {
              "type": "integer"
            },
            {
              "type": "null"
            }
          ],
          "default": null,
          "title": "Active"
        },
        "level": {
          "anyOf": [
            {
              "type": "integer"
            },
            {
              "type": "null"
            }
          ],
          "default": null,
          "title": "Level"
        }
      },
      "title": "metadata_NestedModel",
      "type": "object"
    }
  },
  "properties": {
    "id": {
      "anyOf": [
        {
          "type": "integer"
        },
        {
          "type": "null"
        }
      ],
      "default": null,
      "title": "Id"
    },
    "name": {
      "anyOf": [
        {},
        {
          "type": "null"
        }
      ],
      "default": null,
      "title": "Name"
    },
    "score": {
      "anyOf":

/var/folders/km/x299zwkn69d9gp8bjngqydw80000gn/T/ipykernel_6026/1047736500.py:2: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  print(Model.schema_json(indent=2))


## 🧾 Create and inspect an instance using the inferred model

In [7]:
instance = Model(
    id=4,
    name="Diana",
    score=99.1,
    metadata={"level": 7, "active": True}
)
print(instance)

id=4 name='Diana' score=99.1 metadata=metadata_NestedModel(active=1, level=7)


## Generate Class Code

In [8]:
print(generate_class_code(Model, model_name="UserWithArrow"))

# generated by datamodel-codegen:
#   filename:  schema.json
#   timestamp: 2025-07-07T00:11:28+00:00

from __future__ import annotations

from typing import Any, Optional

from pydantic import BaseModel, Field


class MetadataNestedModel(BaseModel):
    active: Optional[int] = Field(None, title='Active')
    level: Optional[int] = Field(None, title='Level')


class UserWithArrow(BaseModel):
    id: Optional[int] = Field(None, title='Id')
    name: Any = Field(None, title='Name')
    score: Optional[float] = Field(None, title='Score')
    metadata: Optional[MetadataNestedModel] = None

